# Detecting and Handling Outliers

**Module 4: Data Cleaning & Transformation**

## Learning Objectives
- Understand what outliers are and why they matter
- Apply statistical methods to detect outliers (IQR, Z-score)
- Visualize outliers effectively
- Make informed decisions about handling outliers

## Business Context
> "Not all outliers are errors. Some are the most interesting data points in your dataset!"

As a Data Analyst, you need to **identify** outliers, **understand** their nature, and **decide** what to do with them based on business context.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set display options
pd.set_option('display.max_columns', None)
np.random.seed(42)

print("✓ Libraries loaded successfully")

---
## 1. What Are Outliers?

**Outliers** are data points that significantly differ from other observations.

### Types of Outliers

| Type | Description | Example | Action |
|------|-------------|---------|--------|
| **Data Entry Error** | Typo or mistake | Age: 250 instead of 25 | Fix or remove |
| **Measurement Error** | Faulty equipment | Sensor malfunction | Remove |
| **Natural Outlier** | Genuine extreme value | CEO salary in employee data | Keep, analyze separately |
| **Sampling Error** | Wrong population sampled | Luxury car in economy survey | Remove |

### 🎯 Key Business Questions

Before handling outliers, ask:
1. Is this a data quality issue or a genuine extreme value?
2. Will this outlier distort my analysis?
3. Is the outlier itself the insight we're looking for?

In [ ]:
# Create a realistic sales dataset with various types of outliers
np.random.seed(42)
n = 200

sales_data = pd.DataFrame({
    'transaction_id': range(1, n + 1),
    'salesperson': np.random.choice(['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'], n),
    'region': np.random.choice(['North', 'South', 'East', 'West'], n),
    'sale_amount': np.concatenate([
        np.random.normal(500, 150, n - 10),  # Regular sales
        np.array([2500, 3000, 2800, 15000, -500, 50000, 4500, 3200, 2900, 0])  # Outliers
    ]),
    'quantity': np.random.randint(1, 20, n),
    'customer_age': np.concatenate([
        np.random.normal(40, 12, n - 5),
        np.array([250, 15, 95, 105, -5])  # Some are errors, some are valid
    ])
})

# Shuffle the data
sales_data = sales_data.sample(frac=1).reset_index(drop=True)

print("Sales Dataset Created")
print(f"Shape: {sales_data.shape}")
print("\nBasic Statistics:")
print(sales_data[['sale_amount', 'customer_age']].describe())

---
## 2. Visualizing Outliers

Before applying statistical methods, **always visualize** your data first!

In [ ]:
# Visualization methods for outlier detection
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. Box Plot - Classic outlier visualization
axes[0, 0].boxplot(sales_data['sale_amount'])
axes[0, 0].set_title('Box Plot: Sale Amount')
axes[0, 0].set_ylabel('Amount ($)')

# 2. Histogram - Shows distribution
axes[0, 1].hist(sales_data['sale_amount'], bins=30, edgecolor='black')
axes[0, 1].set_title('Histogram: Sale Amount')
axes[0, 1].set_xlabel('Amount ($)')

# 3. Scatter Plot - Shows individual points
axes[0, 2].scatter(range(len(sales_data)), sales_data['sale_amount'], alpha=0.5)
axes[0, 2].set_title('Scatter Plot: Sale Amount')
axes[0, 2].set_xlabel('Index')
axes[0, 2].set_ylabel('Amount ($)')

# Same for customer_age
axes[1, 0].boxplot(sales_data['customer_age'])
axes[1, 0].set_title('Box Plot: Customer Age')

axes[1, 1].hist(sales_data['customer_age'], bins=30, edgecolor='black')
axes[1, 1].set_title('Histogram: Customer Age')

axes[1, 2].scatter(range(len(sales_data)), sales_data['customer_age'], alpha=0.5)
axes[1, 2].set_title('Scatter Plot: Customer Age')

plt.tight_layout()
plt.show()

print("👀 What patterns do you notice? Which points seem unusual?")

---
## 3. Statistical Methods for Outlier Detection

### 3.1 IQR Method (Interquartile Range)

The IQR method identifies outliers as values outside:
- **Lower bound**: Q1 - 1.5 × IQR
- **Upper bound**: Q3 + 1.5 × IQR

**When to use:** Works well for skewed distributions, doesn't assume normality.

In [ ]:
def detect_outliers_iqr(data, column, multiplier=1.5):
    """
    Detect outliers using the IQR method.
    
    Parameters:
    - data: DataFrame
    - column: Column name to check
    - multiplier: IQR multiplier (1.5 = standard, 3 = extreme outliers only)
    
    Returns:
    - DataFrame with outlier information
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    print(f"=== IQR Analysis for '{column}' ===")
    print(f"Q1 (25th percentile): {Q1:.2f}")
    print(f"Q3 (75th percentile): {Q3:.2f}")
    print(f"IQR: {IQR:.2f}")
    print(f"Lower bound: {lower_bound:.2f}")
    print(f"Upper bound: {upper_bound:.2f}")
    print(f"\nOutliers found: {len(outliers)} ({len(outliers)/len(data)*100:.1f}%)")
    
    return outliers, lower_bound, upper_bound

# Apply to sale_amount
outliers_sales, lb_sales, ub_sales = detect_outliers_iqr(sales_data, 'sale_amount')
print("\nOutlier values:")
print(outliers_sales[['transaction_id', 'salesperson', 'sale_amount']].sort_values('sale_amount'))

In [ ]:
# Visualize IQR bounds
fig, ax = plt.subplots(figsize=(12, 5))

# Scatter all points
colors = ['red' if x < lb_sales or x > ub_sales else 'blue' 
          for x in sales_data['sale_amount']]
ax.scatter(range(len(sales_data)), sales_data['sale_amount'], c=colors, alpha=0.6)

# Add boundary lines
ax.axhline(y=lb_sales, color='green', linestyle='--', label=f'Lower bound: {lb_sales:.0f}')
ax.axhline(y=ub_sales, color='green', linestyle='--', label=f'Upper bound: {ub_sales:.0f}')

ax.set_xlabel('Transaction Index')
ax.set_ylabel('Sale Amount ($)')
ax.set_title('Outlier Detection using IQR Method')
ax.legend()
plt.show()

print("🔴 Red points = Outliers | 🔵 Blue points = Normal")

### 3.2 Z-Score Method

The Z-score tells us how many standard deviations a value is from the mean.

$$Z = \frac{x - \mu}{\sigma}$$

**Common thresholds:**
- |Z| > 2: Potential outlier
- |Z| > 3: Likely outlier

**When to use:** Best for normally distributed data.

In [ ]:
def detect_outliers_zscore(data, column, threshold=3):
    """
    Detect outliers using the Z-score method.
    
    Parameters:
    - data: DataFrame
    - column: Column name to check
    - threshold: Z-score threshold (typically 2 or 3)
    
    Returns:
    - DataFrame with outlier information
    """
    mean = data[column].mean()
    std = data[column].std()
    
    # Calculate Z-scores
    z_scores = (data[column] - mean) / std
    
    # Identify outliers
    outliers = data[abs(z_scores) > threshold].copy()
    outliers['z_score'] = z_scores[abs(z_scores) > threshold]
    
    print(f"=== Z-Score Analysis for '{column}' ===")
    print(f"Mean: {mean:.2f}")
    print(f"Std Dev: {std:.2f}")
    print(f"Threshold: |Z| > {threshold}")
    print(f"\nOutliers found: {len(outliers)} ({len(outliers)/len(data)*100:.1f}%)")
    
    return outliers, z_scores

# Apply to sale_amount
outliers_z, z_scores = detect_outliers_zscore(sales_data, 'sale_amount', threshold=3)
print("\nOutlier values with Z-scores:")
print(outliers_z[['transaction_id', 'salesperson', 'sale_amount', 'z_score']].sort_values('z_score'))

In [ ]:
# Compare IQR vs Z-score methods
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# IQR method
sales_data['is_outlier_iqr'] = (sales_data['sale_amount'] < lb_sales) | (sales_data['sale_amount'] > ub_sales)
axes[0].scatter(range(len(sales_data)), sales_data['sale_amount'], 
                c=sales_data['is_outlier_iqr'].map({True: 'red', False: 'blue'}), alpha=0.6)
axes[0].set_title(f'IQR Method: {sales_data["is_outlier_iqr"].sum()} outliers')
axes[0].set_xlabel('Index')
axes[0].set_ylabel('Sale Amount ($)')

# Z-score method
sales_data['is_outlier_z'] = abs(z_scores) > 3
axes[1].scatter(range(len(sales_data)), sales_data['sale_amount'], 
                c=sales_data['is_outlier_z'].map({True: 'red', False: 'blue'}), alpha=0.6)
axes[1].set_title(f'Z-Score Method (|Z|>3): {sales_data["is_outlier_z"].sum()} outliers')
axes[1].set_xlabel('Index')
axes[1].set_ylabel('Sale Amount ($)')

plt.tight_layout()
plt.show()

print("\n💡 Note: Different methods may identify different outliers!")
print("   Choose based on your data distribution.")

---
## 4. Handling Outliers

### Decision Framework

```
Is it a data entry/measurement error?
├── YES → Fix (if possible) or Remove
└── NO → Is it a genuine extreme value?
    ├── YES → Keep but analyze separately OR cap/transform
    └── UNSURE → Investigate further, consult domain expert
```

### 4.1 Removing Outliers

In [ ]:
# Example: Remove clear data errors
print("=== Identifying Data Errors in customer_age ===")

# Impossible ages (negative or > 120)
age_errors = sales_data[(sales_data['customer_age'] < 0) | (sales_data['customer_age'] > 120)]
print(f"\nImpossible ages found: {len(age_errors)}")
print(age_errors[['transaction_id', 'customer_age']])

# Remove these clear errors
sales_cleaned = sales_data[
    (sales_data['customer_age'] >= 0) & 
    (sales_data['customer_age'] <= 120)
].copy()

print(f"\n✓ Removed {len(sales_data) - len(sales_cleaned)} records with impossible ages")
print(f"Dataset size: {len(sales_data)} → {len(sales_cleaned)}")

### 4.2 Capping (Winsorization)

Replace outliers with boundary values. Preserves sample size while reducing impact.

In [ ]:
def cap_outliers(data, column, lower_percentile=5, upper_percentile=95):
    """
    Cap outliers at specified percentiles.
    """
    lower = data[column].quantile(lower_percentile / 100)
    upper = data[column].quantile(upper_percentile / 100)
    
    capped = data[column].clip(lower=lower, upper=upper)
    
    changes = (data[column] != capped).sum()
    print(f"Capped {changes} values to range [{lower:.2f}, {upper:.2f}]")
    
    return capped

# Apply capping to sale_amount
sales_cleaned['sale_amount_capped'] = cap_outliers(
    sales_cleaned, 'sale_amount', 
    lower_percentile=1, upper_percentile=99
)

# Compare distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(sales_cleaned['sale_amount'], bins=30, edgecolor='black')
axes[0].set_title('Original Sale Amount')

axes[1].hist(sales_cleaned['sale_amount_capped'], bins=30, edgecolor='black', color='green')
axes[1].set_title('Capped Sale Amount (1st-99th percentile)')

plt.tight_layout()
plt.show()

### 4.3 Transformation (Log, Square Root)

Reduce the impact of outliers while keeping them in the dataset.

In [ ]:
# Log transformation (for positive values only)
# First, handle negative/zero values
positive_sales = sales_cleaned[sales_cleaned['sale_amount'] > 0].copy()

positive_sales['sale_amount_log'] = np.log(positive_sales['sale_amount'])
positive_sales['sale_amount_sqrt'] = np.sqrt(positive_sales['sale_amount'])

# Visualize transformations
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(positive_sales['sale_amount'], bins=30, edgecolor='black')
axes[0].set_title('Original (Right-Skewed)')
axes[0].set_xlabel('Sale Amount')

axes[1].hist(positive_sales['sale_amount_log'], bins=30, edgecolor='black', color='orange')
axes[1].set_title('Log Transformation')
axes[1].set_xlabel('log(Sale Amount)')

axes[2].hist(positive_sales['sale_amount_sqrt'], bins=30, edgecolor='black', color='purple')
axes[2].set_title('Square Root Transformation')
axes[2].set_xlabel('√Sale Amount')

plt.tight_layout()
plt.show()

print("💡 Transformations can make data more normally distributed!")

### 4.4 Keeping Outliers (Separate Analysis)

Sometimes outliers are the most valuable data points!

In [ ]:
# Identify high-value sales (potential VIP customers or major deals)
threshold_high = sales_data['sale_amount'].quantile(0.95)

high_value_sales = sales_data[sales_data['sale_amount'] > threshold_high]
regular_sales = sales_data[sales_data['sale_amount'] <= threshold_high]

print(f"=== Segmenting by Sale Value ===")
print(f"\nHigh-Value Sales (top 5%): {len(high_value_sales)} transactions")
print(f"Regular Sales: {len(regular_sales)} transactions")

print(f"\n--- High-Value Sales Summary ---")
print(high_value_sales[['transaction_id', 'salesperson', 'sale_amount', 'region']].sort_values('sale_amount', ascending=False))

print(f"\n--- Average Sale by Segment ---")
print(f"High-Value: ${high_value_sales['sale_amount'].mean():,.2f}")
print(f"Regular: ${regular_sales['sale_amount'].mean():,.2f}")

---
## 5. Impact Analysis: Before vs After

Always document how outlier handling affects your analysis!

In [ ]:
def outlier_impact_report(original, cleaned, column):
    """
    Generate a report showing the impact of outlier removal.
    """
    print("=" * 50)
    print(f"OUTLIER IMPACT REPORT: {column}")
    print("=" * 50)
    
    metrics = ['count', 'mean', 'median', 'std', 'min', 'max']
    
    print(f"\n{'Metric':<10} {'Original':>15} {'After Cleaning':>15} {'Change':>12}")
    print("-" * 55)
    
    for metric in metrics:
        orig_val = getattr(original[column], metric)()
        clean_val = getattr(cleaned[column], metric)()
        
        if metric == 'count':
            change = f"{clean_val - orig_val:+.0f}"
            print(f"{metric:<10} {orig_val:>15.0f} {clean_val:>15.0f} {change:>12}")
        else:
            pct_change = ((clean_val - orig_val) / orig_val * 100) if orig_val != 0 else 0
            change = f"{pct_change:+.1f}%"
            print(f"{metric:<10} {orig_val:>15.2f} {clean_val:>15.2f} {change:>12}")

# Apply to our data
# Remove extreme outliers for comparison
Q1 = sales_data['sale_amount'].quantile(0.25)
Q3 = sales_data['sale_amount'].quantile(0.75)
IQR = Q3 - Q1
clean_for_report = sales_data[
    (sales_data['sale_amount'] >= Q1 - 1.5 * IQR) & 
    (sales_data['sale_amount'] <= Q3 + 1.5 * IQR)
]

outlier_impact_report(sales_data, clean_for_report, 'sale_amount')

---
## 6. Practical Exercises

### Exercise 1: Detect Outliers in Employee Data

In [ ]:
# Employee salary data
np.random.seed(42)
n = 100

employees = pd.DataFrame({
    'employee_id': range(1, n + 1),
    'department': np.random.choice(['Sales', 'IT', 'HR', 'Marketing'], n),
    'salary': np.concatenate([
        np.random.normal(55000, 10000, n - 5),
        np.array([150000, 200000, 25000, 300000, 15000])  # Outliers
    ]),
    'years_experience': np.random.randint(1, 20, n)
})

employees = employees.sample(frac=1).reset_index(drop=True)

print("Employee Data:")
employees.head(10)

In [ ]:
# TODO: Visualize salary distribution with a boxplot and histogram


In [ ]:
# TODO: Use IQR method to detect salary outliers
# Use the detect_outliers_iqr() function


In [ ]:
# TODO: Use Z-score method to detect salary outliers
# Compare with IQR results


### Exercise 2: Handle Outliers Appropriately

In [ ]:
# TODO: For the employee data above, decide how to handle each outlier:
# - $15,000 and $25,000 (very low salaries)
# - $150,000, $200,000, $300,000 (very high salaries)

# Questions to consider:
# 1. Are these data errors or genuine values?
# 2. How does experience relate to these outliers?
# 3. Should you remove, cap, or keep them?


In [ ]:
# TODO: Apply your chosen strategy and compare statistics before/after


### Exercise 3: Business Context Decision

In [ ]:
# E-commerce transaction data
transactions = pd.DataFrame({
    'order_id': range(1, 201),
    'order_value': np.concatenate([
        np.random.exponential(50, 190),  # Most orders are small
        np.array([500, 800, 1200, 50000, 75000, 100000, 2500, 3000, 4500, 5000])  # Various sizes
    ]),
    'customer_type': np.random.choice(['Individual', 'Business'], 200, p=[0.8, 0.2])
})

print("Transaction Data Summary:")
print(transactions['order_value'].describe())

In [ ]:
# TODO: This is a business decision exercise
# 
# Scenario: Your manager asks you to calculate the "average order value" 
# for a marketing report.
#
# 1. What are the outliers?
# 2. Should you include the $50,000-$100,000 orders?
# 3. How would you present this to stakeholders?
# 4. Would your decision change if you knew the business sells both 
#    consumer products AND B2B bulk orders?

# Write your analysis and recommendation below:


---
## 7. Key Takeaways

### ✅ Best Practices

1. **Always visualize first** - boxplots, histograms, scatter plots
2. **Use appropriate methods** - IQR for skewed data, Z-score for normal data
3. **Consider the business context** - not all outliers should be removed
4. **Document your decisions** - explain why you kept or removed values
5. **Report the impact** - show how outlier handling affected your analysis

### ⚠️ Common Mistakes to Avoid

1. Automatically removing all statistical outliers
2. Not checking if outliers are data errors vs. genuine values
3. Using Z-score on highly skewed data
4. Forgetting to document your decisions

### 🎯 Decision Checklist

Before handling any outlier, ask:
- [ ] Is this value physically/logically possible?
- [ ] Could this be a data entry error?
- [ ] Is this outlier important for my analysis?
- [ ] How will removing/keeping it affect my results?
- [ ] Can I explain my decision to a stakeholder?